## Darts benchamrk 

This notebook showcases tools developed for benchmarking the different forecasting models in Darts, which could be useful for other developers.
We will showcase:

**Model evaluation**: A wrapper around backtest and historical_forecasts to standardize these function for all models
**Auto-ML with Optuna and raytune**: hyperparameter tuning with predefined broad parameter space for all model supported
**Experiment**: Function to perform cross comparison of a list of models on a list of datasets

In [ ]:
#@title Install dependencies
!pip install -q darts==0.24 ray==2.3.1 optuna==3.1.0

files = [
    "benchmark_tools.py",
    "model_evaluation.py",
    "optuna_search.py",
    "param_space.py",
]
for f in files:
    url = f"https://raw.githubusercontent.com/unit8co/darts/docs/benchmarks/experiments/{f}"
    !wget --no-cache --backups=1 {url}

from benchmark_tools import Dataset, experiment, silence_prompt
from darts.models import NaiveSeasonal, NHiTSModel, Prophet
from darts.utils import missing_values

models = [NaiveSeasonal, NHiTSModel, Prophet]

In [ ]:
from darts.datasets import AirPassengersDataset, WeatherDataset

air_pass_series = missing_values.fill_missing_values(AirPassengersDataset().load())
weather_ds = missing_values.fill_missing_values(WeatherDataset().load().resample("1h"))[
    -1500:
]
weather_past_cov = weather_ds[
    ["p (mbar)", "wv (m/s)", "wd (deg)", "rain (mm)", "raining (s)", "SWDR (W/m²)"]
]
weather_series = weather_ds["T (degC)"]

dataset_air_passengers = Dataset(name="Air passengers", series=air_pass_series)
dataset_weather = Dataset(
    name="Weather", series=weather_series, past_covariates=weather_past_cov
)

In [ ]:
from optuna_search import optuna_search

silence_prompt()
best_params = optuna_search(
    model_class=Prophet, **dataset_air_passengers._asdict(), time_budget=60
)
model = Prophet(**best_params)


forecasts = model.historical_forecasts(series=dataset_air_passengers.series, start=0.8)

dataset_air_passengers.series.plot(label="actual")
forecasts.plot(label="prediction")

In [ ]:
results = experiment(
    list_datasets=[dataset_air_passengers, dataset_weather],
    models=models,
    grid_search=False,
    forecast_horizon=0.05, # the forecast horizon will be set to 5% of the test set length
    repeat=3,
    silent_search=True,
)

print("\n\n", results)